# Task 2: General Hartree-Fock solver

Having set up an initial single-particle basis set $\{\chi_{\alpha}\}_{\alpha = 1}^{l}$ we now set out to find the ground state of the two-body problem.
The full Hamiltonian used by {cite}`zanghellini_2004` is given by
\begin{align}
    \hat{H}
    = \hat{h} + \hat{u}
    = \sum_{i = 1}^{n}\left(
        -\frac{1}{2}\frac{\text{d}^2}{\text{d}x^2}
        + \frac{1}{2}\omega^2 x^2
    \right)
    + \sum_{i = 1}^{n} \sum_{j = i + 1}^{n} \frac{\alpha}{\sqrt{(x_i - x_j)^2 + a^2}},
\end{align}
where the number of particles $n = 2$, $a = 0.25$, $\alpha = 1$ (not to be confused with the index $\alpha$ in the basis of harmonic oscillator eigenstates), and $\omega = 0.25$.

## Types of orbital systems

In quantum chemistry it is common to call the initial single-particle functions for _atomic orbitals_.
We will also adhere to this convention, thus making the basis of harmonic oscillator single-particle functions the atomic orbitals of our system.
Now, as the harmonic oscillator Hamiltonian is independent of spin we have restricted our view to the spatial part of the atomic orbitals when using quantum-systems to construct our basis elements.
